In [1]:
from tensorflow.keras.models import load_model
import librosa
import pygame
import numpy as np
import time
import json
from uuid import uuid4
import requests

pygame 2.4.0 (SDL 2.26.4, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pyaudio
import numpy as np

def record_audio(duration, sample_rate):
    chunk_size = 1024
    audio_buffer = []

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=chunk_size)

    num_frames = int(duration * sample_rate / chunk_size)
    for _ in range(num_frames):
        data = stream.read(chunk_size)
        audio_buffer.append(np.frombuffer(data, dtype=np.int16))

    stream.stop_stream()
    stream.close()
    p.terminate()

    audio_data = np.concatenate(audio_buffer)
    return audio_data

# Hangrögzítés 4 másodpercig, 44100 mintavételezési ráta mellett
duration = 4  # Másodpercekben
sample_rate = 22050  # Mintavételezési ráta

model = load_model('./wake_wordv1.h5')
np.set_printoptions(suppress=True)

In [3]:
import librosa
import librosa.util
import numpy as np

def preprocess_log_mel_spectrogram_pyaudio(audio_data, sr, n_mels=128, duration=4.0):
    # Convert audio data to floating-point
    audio_data = librosa.util.buf_to_float(audio_data)

    hop_length = int(sr * (0.02 - 0.01))
    # Extract the Log-Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(audio_data, sr=sr, n_mels=n_mels, hop_length=hop_length)
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

    return log_mel_spectrogram

In [35]:
list = [1,2,3,4,5,6,7,8,9,1,2,3]

In [56]:
class Command:
    def __init__(self, commandSentense):
        self.id = str(uuid4())
        self.commandSentense = commandSentense

def wakeworddet():

    audio_data = record_audio(3.0, sample_rate)
    spec_to_predict = preprocess_log_mel_spectrogram_pyaudio(audio_data, 22050)
    desired_shape = model.input_shape

    # Kiszámítjuk a szükséges padding-et az első két dimenzióra
    pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

    # Padding-et alkalmazunk a spektrogramra
    padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')


    spec_to_predict = np.expand_dims(padded_spec, axis=-1)


    spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)

    predictions = model.predict(spec_to_predict_batch)
    print(predictions)



    if predictions > .95:
        payload = Command("wakeword")
        url = "http://localhost:5025/Command"
        headers = {'Content-Type': 'application/json'}
        data = json.dumps(payload.__dict__)
        response = requests.post(url, headers=headers, data=data)
        print('wakeword')
    else:
        print('--------------')
    

In [ ]:

import threading
import time



while True:    
    szal = threading.Thread(target=wakeworddet)
    szal.start()
    time.sleep(3)